<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Assignment_2_iNaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as L
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping,Callback,ModelCheckpoint

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Deep Learning/inaturalist_12K/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Deep Learning/inaturalist_12K


**Loading & Splitting Dataset**

In [4]:
transform = transforms.Compose(
    [ transforms.Resize((224, 224)),
      transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = ImageFolder(root='train', transform=transform)
input_dims = (train_dataset[0][0].size()[1],train_dataset[0][0].size()[2],train_dataset[0][0].size(0))
print(f'Input image shape ({input_dims[0]},{input_dims[1]},{input_dims[2]})')
print('-------------')

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size


train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
test_dataset = ImageFolder(root='val',transform=transform)


class_names = train_dataset.dataset.classes
class_counts = {class_name: 0 for class_name in class_names}
for _, class_idx in train_dataset.dataset.samples:
    class_name = class_names[class_idx]
    class_counts[class_name] += 1

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, No. of samples: {count}")

num_classes = len(class_counts.keys())
print('-------------')
print(f'No of classes {num_classes}')
print(f'Train Samples {len(train_dataset)}')
print(f'Val Samples {len(val_dataset)}')
print(f'Test Samples {len(test_dataset)}')

Input image shape (224,224,3)
-------------
Class: Amphibia, No. of samples: 1010
Class: Animalia, No. of samples: 1000
Class: Arachnida, No. of samples: 1001
Class: Aves, No. of samples: 1000
Class: Fungi, No. of samples: 999
Class: Insecta, No. of samples: 1010
Class: Mammalia, No. of samples: 1000
Class: Mollusca, No. of samples: 1010
Class: Plantae, No. of samples: 1000
Class: Reptilia, No. of samples: 1022
-------------
No of classes 10
Train Samples 8041
Val Samples 2011
Test Samples 2001


In [5]:
batch_size=8
NUM_WORKERS = int(os.cpu_count() / 2)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# def imshow(img):
#     img = img / 2 + 0.5
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# dataiter = iter(train_dataloader)
# images, labels = next(dataiter)


# imshow(torchvision.utils.make_grid(images))

In [9]:
class Net(nn.Module):
  def __init__(self,
                cnn_activation_function,
                fcc_activation_function,
                num_filters,
                filter_size,
                dense_neurons,
                stride=2,
                input_dims=input_dims,
                num_classes=num_classes):
      super().__init__()
      width,height,channels = input_dims
      layers = []
      for i in range(5):
        in_channels = channels if i==0 else num_filters
        layers.extend([nn.Conv2d(in_channels, num_filters, kernel_size=filter_size),
                      getattr(nn, cnn_activation_function)(inplace=True)])
        width = width - filter_size + 1
        height = height - filter_size + 1
        layers.append(nn.MaxPool2d(kernel_size=filter_size,stride=stride))
        width = ((width - filter_size)//stride) + 1
        height = ((height - filter_size)//stride) + 1
      layers.append(nn.Flatten())
      layers.append(nn.Linear(width * height * num_filters, dense_neurons))
      layers.append(getattr(nn, fcc_activation_function)(inplace=True))
      layers.append(nn.Linear(dense_neurons, num_classes))
      layers.append(nn.Softmax())
      self.net = nn.Sequential(*layers)

  def forward(self, x):
    x =  self.net(x)
    return F.log_softmax(x, dim=1)

In [10]:
class MyCnnModel(L.LightningModule):
    def __init__(self,
                 cnn_activation_function = 'ReLU',
                 fcc_activation_function = 'ReLU',
                 loss_function='cross_entropy',
                 num_filters=32,
                 filter_size=3,
                 dense_neurons=1024,
                 learning_rate=0.01,
                 optimizer='Adam'):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net(cnn_activation_function,fcc_activation_function,num_filters,filter_size,dense_neurons)
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = learning_rate
        self.optimizer = optimizer

    def forward(self, x):
        x = self.model(x)
        return F.softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
      loss,acc = self._common_step(batch,batch_idx,'train')
      # self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      return self._common_step(batch,batch_idx,'val')


    def test_step(self,batch,batch_idx):
      return self._common_step(batch,batch_idx,'test')

    def _common_step(self, batch, batch_idx,op_type):
      inputs, target = batch
      output = self.forward(inputs)
      loss = self.loss_function(output, target)
      acc = self.accuracy(output, target)
      self.log(f'{op_type}_loss', loss)
      self.log(f'{op_type}_acc', acc)
      return loss,acc

    def configure_optimizers(self):
      return getattr(torch.optim,self.optimizer)(self.parameters(), lr=self.learning_rate)

In [ ]:
class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")


model = MyCnnModel()
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
trainer = L.Trainer(max_epochs=1,accelerator="auto", limit_train_batches=0.5,
    limit_val_batches=0.5,precision='16-mixed',check_val_every_n_epoch=10,callbacks=[PrintCallback(),checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model    | Net                | 573 K 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
573 K     Trainable params
0         Non-trainable params
573 K     Total params
2.294     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training is started!


Training: |          | 0/? [00:00<?, ?it/s]